# Computer-aided simulation and visualisation of metabolic flux in diabetic cardiomyocytes

### Aim
**Visualise metabolic fluxes** (rate of turnover of a metabolite) to understand how glucose takeup (or lack thereof) affects cardiac output.

*Diabetic people have a higher chance of dying of heart failure than non-diabetic people.*

### Hypothesis

Cardiac specific insulin resistance → less available glucose → less available ATP → lower cardiac output → heart failure.

### Setup

In [1]:
import xml.dom.minidom
import pandas as pd
import cobra
import json
import numpy as np

In [2]:
file = xml.dom.minidom.parse("icardio.xml")

### Dataframe

#### Dataframe makes connection between species names, compartments and IDs

In [3]:
model = file.documentElement
species = model.getElementsByTagName("species")

species_names = []
species_groups = []
species_ids = []

for element in species:
    species_names.append(element.getAttribute("name"))
    species_groups.append(element.getAttribute("compartment"))
    species_ids.append(element.getAttribute("id"))
     
dataframe = pd.DataFrame({"names": species_names, "groups": species_groups, "ids": species_ids})
dataframe.index = np.arange(1, len(dataframe)+1)
dataframe

,names,groups,ids
1,(10Z)-heptadecenoic acid,c,m00003__91__c__93__
2,(10Z)-heptadecenoic acid,r,m00003__91__r__93__
3,(10Z)-heptadecenoic acid,s,m00003__91__s__93__
4,(10Z)-heptadecenoyl-CoA,c,m00004__91__c__93__
5,(10Z)-heptadecenoyl-CoA,m,m00004__91__m__93__
...,...,...,...
2886,hyocholoyl-CoA,c,m90185__91__c__93__
2887,protein,c,m90190__91__c__93__
2888,glycogen storage pool,c,m90194__91__c__93__
2889,intracellular lipids,c,m90195__91__c__93__


### Secondary Dataframe

#### Dataframe makes connection between reactions and species (separated into reactants and products). The dataframe is also used to arrange the information into a format suitable for export.

In [4]:
list_of_reactions = []
reactions = model.getElementsByTagName("reaction")

for reaction in reactions:
    this_reaction = {}
    this_reaction["id"] = reaction.getAttribute("name")
    list_of_reactants = reaction.getElementsByTagName("listOfReactants")
    list_of_products = reaction.getElementsByTagName("listOfProducts")
    for reactant in list_of_reactants:
        this_reaction["Reactants"] = []
        species_reference = reactant.getElementsByTagName("speciesReference")
        for species in species_reference:
            this_reaction["Reactants"].append({"id": species.getAttribute("species"),
                                               "name": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "names"].array[0],
                                               "group": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "groups"].array[0]})
    for product in list_of_products:
        this_reaction["Products"] = []
        species_reference = product.getElementsByTagName("speciesReference")
        for species in species_reference:
            this_reaction["Products"].append({"id": species.getAttribute("species"),
                                               "name": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "names"].array[0],
                                               "group": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "groups"].array[0]})        
        
    list_of_reactions.append(this_reaction)

list_of_reactions[0:5]

[{'id': 'RCR10001',
  'Reactants': [{'id': 'm01285__91__c__93__', 'name': 'ADP', 'group': 'c'}],
  'Products': [{'id': 'm01334__91__c__93__', 'name': 'AMP', 'group': 'c'},
   {'id': 'm01371__91__c__93__', 'name': 'ATP', 'group': 'c'}]},
 {'id': 'RCR10002',
  'Reactants': [{'id': 'm01252__91__c__93__', 'name': 'acetate', 'group': 'c'},
   {'id': 'm01371__91__c__93__', 'name': 'ATP', 'group': 'c'},
   {'id': 'm01597__91__c__93__', 'name': 'CoA', 'group': 'c'}],
  'Products': [{'id': 'm01261__91__c__93__',
    'name': 'acetyl-CoA',
    'group': 'c'},
   {'id': 'm01334__91__c__93__', 'name': 'AMP', 'group': 'c'},
   {'id': 'm02759__91__c__93__', 'name': 'PPi', 'group': 'c'}]},
 {'id': 'RCR10004',
  'Reactants': [{'id': 'm01285__91__m__93__', 'name': 'ADP', 'group': 'm'}],
  'Products': [{'id': 'm01334__91__m__93__', 'name': 'AMP', 'group': 'm'},
   {'id': 'm01371__91__m__93__', 'name': 'ATP', 'group': 'm'}]},
 {'id': 'RCR10005',
  'Reactants': [{'id': 'm01369__91__c__93__',
    'name': 'as

### Selector

#### The selector filters nodes and links according to the different compartments (here it is called a "group"). The resulting information is also organised into a format suitable for export.

Selection was necessary because a graph of the whole dataset is too busy.

In [110]:
nodes = []
links = []
pseudo_nodes_s = []
pseudo_nodes_e = []
selector = "s"
selected_list_of_reactions = []


for index, item in enumerate(species_ids):
    if dataframe.loc[dataframe["ids"] == item, "groups"].array[0] == selector:
        nodes.append({"id": item, "name": species_names[index], "group": species_groups[index]})

for item in list_of_reactions:
    for reactants in item["Reactants"]:
           if reactants["group"] == selector and item not in selected_list_of_reactions:
                selected_list_of_reactions.append(item)
    if "Products" in item:
        for products in item["Products"]:
            if products["group"] == selector and item not in selected_list_of_reactions:
                selected_list_of_reactions.append(item)

for item in selected_list_of_reactions:
    if "Products" in item:
        pseudo_nodes_s.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"})
        pseudo_nodes_e.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
        for reactant in item["Reactants"]:
            links.append({"source": reactant["id"], "target": item["id"] + "s", "arrow": False})
            if reactant not in nodes:
                nodes.append(reactant)
        for product in item["Products"]:
            links.append({"source": item["id"] + "e", "target": product["id"], "arrow": True})
            if product not in nodes:
                nodes.append(product)

for index, item in enumerate(pseudo_nodes_s):
    links.append({"source": item["id"], "target": pseudo_nodes_e[index]["id"], "arrow": False})

all_nodes = nodes + pseudo_nodes_s + pseudo_nodes_e
    
print("This selection has produced: " + str(len(all_nodes)) + " nodes and " + str(len(links)) + " links.")

This selection has produced: 2281 nodes and 4901 links.


### Collate data into format suitable for JSON export and further use

In [6]:
icardio_s_data= {"nodes": [], "links": []}
icardio_s_data["nodes"] += all_nodes
icardio_s_data["links"] += links

### Export

In [7]:
json_string = json.dumps(icardio_s_data)

with open('icardio_s_data.json', 'w', encoding='utf-8') as f:
    json.dump(icardio_s_data, f, ensure_ascii=False, indent=4)

### Pathways

#### This code collects information about the different metabolic pathways specified in the iCardio model.

In [8]:
pathways = model.getElementsByTagName("groups:group")
len(pathways)

for pathway in pathways:
    print(pathway.getAttribute("groups:name"))


Alanine aspartate and glutamate metabolism
Arachidonic acid metabolism
Arginine and proline metabolism
Aromatic amino acid metabolism
Beta oxidation of fatty acids
Bile acid metabolism
Biomass
Carbohydrate metabolism
Central carbon metabolism
Cholesterol metabolism
Cholesterol-ester metabolism
Cysteine and methionine metabolism
Eicosanoid metabolism
Electron transport chain
Exchange
Fatty acid metabolism
Glutathione metabolism
Glycan metabolism
Glycerolipid metabolism
Glycine serine and threonine metabolism
Inositol phosphate metabolism
Lysine metabolism
Miscellaneous
Nucleotide metabolism
Omega-3 fatty acid metabolism
Omega-6 fatty acid metabolism
Pentose phosphate pathway
Porphyrin metabolism
Protein metabolism
Proteoglycan metabolism
Purine metabolism
Pyrimidine metabolism
Serotonin and melatonin biosynthesis
Sphingolipid metabolism
Steroid metabolism
Transport
Valine leucine and isoleucine metabolism
Vitamin A metabolism
Vitamin B metabolism
Vitamin C metabolism
Vitamin D metaboli

#### The below code collects information about all pathways and arranges it into the below format. (Takes about 2 minutes to run)

In [112]:
list_of_pathways = []
for item in pathways:
    if item.hasAttribute("groups:name"):
        name = item.getAttribute("groups:name")
    list_of_members = item.getElementsByTagName("groups:listOfMembers")
    for member in list_of_members:
        member_rxn = member.getElementsByTagName("groups:member")
        name_list = []
        for rxn in member_rxn:
            rxn_item = {}
            rxn_item["id"] = rxn.getAttribute("groups:idRef")
            for item in list_of_reactions:
                if item["id"] == rxn_item["id"]:
                    rxn_item["Reactants"] = item["Reactants"]
                    if "Products" in item:
                        rxn_item["Products"] = item["Products"]
            name_list.append(rxn_item)
        pathway_dict = {}
        pathway_dict["name"] = name
        pathway_dict["Reactions"] = name_list
        list_of_pathways.append(pathway_dict)

print("""{'name': 'Alanine aspartate and glutamate metabolism',
 'Reactions': [{'id': 'RCR10001',
   'Reactants': [{'id': 'm01285__91__c__93__', 'name': 'ADP', 'group': 'c'}],
   'Products': [{'id': 'm01334__91__c__93__', 'name': 'AMP', 'group': 'c'},
    {'id': 'm01371__91__c__93__', 'name': 'ATP', 'group': 'c'}]},""")



{'name': 'Alanine aspartate and glutamate metabolism',
 'Reactions': [{'id': 'RCR10001',
   'Reactants': [{'id': 'm01285__91__c__93__', 'name': 'ADP', 'group': 'c'}],
   'Products': [{'id': 'm01334__91__c__93__', 'name': 'AMP', 'group': 'c'},
    {'id': 'm01371__91__c__93__', 'name': 'ATP', 'group': 'c'}]},


#### Selection of the fatty acid metabolism pathway below.

The fatty acid metabolism pathway was picked because heart muscle cells can use fatty acids as an alternative energy source when ATP is low.

In [111]:
fatty_acid = list_of_pathways[15]

counter = 0
for item in fatty_acid["Reactions"]:
    counter += 1
    
print("Number of reactions in the fatty acid metabolism pathway: " + str(counter))

Number of reactions in the fatty acid metabolism pathway: 4198


#### The below code gathers information about the reactants and products in all reactions of the fatty acid metabolism pathway and organises it into a suitable format.

In [114]:
nodes_FA = []
links_FA = []
links_FA_2 = []
pseudo_nodes_s_FA = []
pseudo_nodes_e_FA = []

for item in fatty_acid["Reactions"]:
    for reactant in item["Reactants"]:
        if reactant not in nodes_FA:
            nodes_FA.append(reactant)
    if "Products" in item:
        pseudo_nodes_s_FA.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"})
        pseudo_nodes_e_FA.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
        for reactant in item["Reactants"]:
            links_FA.append({"source": reactant["id"], "target": item["id"] + "s"})
        for product in item["Products"]:
            links_FA.append({"source": item["id"] + "e", "target": product["id"]})
            if product not in nodes_FA:
                nodes_FA.append(product)
for item in pseudo_nodes_s_FA:
    links_FA_2.append({"source": item["id"], "target": item["id"].replace("s", "e")})

all_nodes_FA = nodes_FA + pseudo_nodes_s_FA + pseudo_nodes_e_FA
all_links_FA = links_FA + links_FA_2

print("There are " + str(len(nodes_FA)) + " individual species, " + str(len(pseudo_nodes_s_FA)) + " reactions that have products, and " + str(len(all_links_FA)) + " links will be shown between the reaction participants.")

There are 2890 individual species, 4042 reactions that have products, and 22663 links will be shown between the reaction participants.


### Collate data into format suitable for JSON export and further use

In [13]:
icardio_fatty_acid= {"nodes": [], "links": []}
icardio_fatty_acid["nodes"] += all_nodes_FA
icardio_fatty_acid["links"] += all_links_FA

### Export

In [14]:
json_string = json.dumps(icardio_fatty_acid)

with open('icardio_fatty_acid.json', 'w', encoding='utf-8') as f:
    json.dump(icardio_fatty_acid, f, ensure_ascii=False, indent=4)

### COnstraint-Based Reconstruction and Analysis (COBRA)

In [15]:
wildtype = cobra.io.read_sbml_model("icardio.xml")
optim_dataset = wildtype.optimize()
optim_dataset

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11


,fluxes,reduced_costs
RCR10001,736.646705,0.000000
RCR10002,0.000000,-0.333330
RCR10004,0.000000,0.000000
RCR10005,70.938840,0.000000
RCR10006,32.741003,0.000000
...,...,...
RCR90207,0.000000,0.000000
RCR90208,0.000000,-0.583327
RCR90209,0.000000,0.000000
RCR90210,-16.571649,0.000000


In [31]:
print(wildtype.objective.expression)

1.0*RCR90143 - 1.0*RCR90143_reverse_0dca8


In [33]:
wildtype.objective = "RCR20085"

In [34]:
print(wildtype.objective.expression)

1.0*RCR20085 - 1.0*RCR20085_reverse_38999


In [116]:
print(wildtype.objective.direction)

max


In [35]:
RCR20085_dataset = wildtype.optimize()
RCR20085_dataset

,fluxes,reduced_costs
RCR10001,736.646705,0.0
RCR10002,0.000000,0.0
RCR10004,0.000000,0.0
RCR10005,70.938840,0.0
RCR10006,32.741003,0.0
...,...,...
RCR90207,0.000000,0.0
RCR90208,0.000000,-0.0
RCR90209,0.000000,0.0
RCR90210,-16.571649,-0.0


In [36]:
negative_colours = ["#0000ff", "#7145ff", "#a074ff", "#c4a2ff", "#e3d0ff", "#ffffff"] #blue -> white

positive_colours = ["#ffffff", "#ffd9cb", "#ffb299", "#ff8969", "#ff5b3a", "#ff0000"] #white -> red

rxn_names = [_.name for _ in wildtype.reactions]
rxn_wt_flux = [_.flux for _ in wildtype.reactions]

colours = []
for flux in rxn_wt_flux:
    if -1000 <= flux <= -800:
        colours.append("#0000ff")
    elif -800 < flux <= -600:
        colours.append("#7145ff")
    elif -600 < flux <= -400:
        colours.append("#a074ff")
    elif -400 < flux <= -200:
        colours.append("#c4a2ff")
    elif -200 < flux < 0:
        colours.append("#e3d0ff")
    elif flux == 0:
        colours.append("#ffffff")
    elif 0 < flux <= 200:
        colours.append("#ffd9cb")
    elif 200 < flux <= 400:
        colours.append("#ffb299")
    elif 400 < flux <= 600:
        colours.append("#ff8969")
    elif 600 < flux <= 800:
        colours.append("#ff5b3a")
    elif 800 < flux <= 1000:
        colours.append("#ff0000")

print(len(colours))

4200


In [37]:
reaction_ids = []
for reaction in reactions:
    reaction_ids.append(reaction.getAttribute("name"))
    
print(len(reaction_ids))

asdf = pd.DataFrame({"fluxes": rxn_wt_flux, "colours": colours})
asdf.index = reaction_ids
asdf

4200


,fluxes,colours
RCR10001,736.646705,#ff5b3a
RCR10002,0.000000,#ffffff
RCR10004,0.000000,#ffffff
RCR10005,70.938840,#ffd9cb
RCR10006,32.741003,#ffd9cb
...,...,...
RCR90207,0.000000,#ffffff
RCR90208,0.000000,#ffffff
RCR90209,0.000000,#ffffff
RCR90210,-16.571649,#e3d0ff


In [62]:
nodes_FA2 = []
links_FA2 = []
pseudo_nodes_s_FA2 = []
pseudo_nodes_e_FA2 = []
selector_FA2 = "s"
selected_list_of_reactions_FA2 = []

for item in nodes_FA:
    if item["group"] == selector_FA2:
        nodes_FA2.append(item)

for item in fatty_acid["Reactions"]:
    for reactant in item["Reactants"]:
        if reactant["group"] == selector_FA2 and item not in selected_list_of_reactions_FA2:
                selected_list_of_reactions_FA2.append(item)
    if "Products" in item:
        for product in item["Products"]:
            if product["group"] == selector_FA2 and item not in selected_list_of_reactions_FA2:
                selected_list_of_reactions_FA2.append(item)

for item in selected_list_of_reactions_FA2:
    if "Products" in item:
        pseudo_nodes_s_FA2.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"})
        pseudo_nodes_e_FA2.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
        for reactant in item["Reactants"]:
            links_FA2.append({"source": reactant["id"], "target": item["id"] + "s", "arrow": False, "colour": asdf.loc[item["id"], "colours"]})
            if reactant not in nodes_FA2:
                nodes_FA2.append(reactant)
        for product in item["Products"]:
            links_FA2.append({"source": item["id"] + "e", "target": product["id"], "arrow": True, "colour": asdf.loc[item["id"], "colours"]})
            if product not in nodes_FA2:
                nodes_FA2.append(product)

for index, item in enumerate(pseudo_nodes_s_FA2):
    links_FA2.append({"source": item["id"], "target": pseudo_nodes_e_FA2[index]["id"], "arrow": False, "colour": asdf.loc[item["id"].replace("s", "").replace("e", ""), "colours"]})

all_nodes_FA2 = nodes_FA2 + pseudo_nodes_s_FA2 + pseudo_nodes_e_FA2

print(len(nodes_FA2))
print(len(pseudo_nodes_s_FA2))
print(len(pseudo_nodes_e_FA2))
print(len(all_nodes_FA2))
print(len(links_FA2))
print(len(selected_list_of_reactions_FA2))


505
888
888
2281
4901
1041


In [63]:
icardio_fatty_acid_s= {"nodes": [], "links": []}
icardio_fatty_acid_s["nodes"] += all_nodes_FA2
icardio_fatty_acid_s["links"] += links_FA2

In [64]:
json_string = json.dumps(icardio_fatty_acid_s)

with open('icardio_fatty_acid_s.json', 'w', encoding='utf-8') as f:
    json.dump(icardio_fatty_acid_s, f, ensure_ascii=False, indent=4)

### Generate perturbed data

In [67]:
perturbed = cobra.io.read_sbml_model("icardio.xml")
print(perturbed.objective.expression)

1.0*RCR90143 - 1.0*RCR90143_reverse_0dca8


In [68]:
perturbed.objective = "RCR20085"
print(perturbed.objective.expression)

1.0*RCR20085 - 1.0*RCR20085_reverse_38999


In [69]:
glc_c_to_s = perturbed.reactions.get_by_id("RCR41035")
glc_s_to_c = perturbed.reactions.get_by_id("RCR40464")
glc_c_to_g = perturbed.reactions.get_by_id("RCR20130")
glc_c_to_r = perturbed.reactions.get_by_id("RCR20071")

In [70]:
glc_c_to_s.lower_bound, glc_c_to_s.upper_bound = 0, 0
glc_s_to_c.lower_bound, glc_c_to_s.upper_bound = 0, 0
glc_c_to_g.lower_bound, glc_c_to_s.upper_bound = 0, 0
glc_c_to_r.lower_bound, glc_c_to_s.upper_bound = 0, 0

In [71]:
perturbed.optimize()

,fluxes,reduced_costs
RCR10001,0.0,-0.0
RCR10002,0.0,0.0
RCR10004,-500.0,-0.0
RCR10005,0.0,0.0
RCR10006,0.0,0.0
...,...,...
RCR90207,0.0,0.0
RCR90208,0.0,0.0
RCR90209,0.0,0.0
RCR90210,0.0,-0.0


In [72]:
perturbed_rxn_names = [_.name for _ in perturbed.reactions]
perturbed_flux = [_.flux for _ in perturbed.reactions]

colours = []
for flux in perturbed_flux:
    if -1000 <= flux <= -800:
        colours.append("#0000ff")
    elif -800 < flux <= -600:
        colours.append("#7145ff")
    elif -600 < flux <= -400:
        colours.append("#a074ff")
    elif -400 < flux <= -200:
        colours.append("#c4a2ff")
    elif -200 < flux < 0:
        colours.append("#e3d0ff")
    elif flux == 0:
        colours.append("#ffffff")
    elif 0 < flux <= 200:
        colours.append("#ffd9cb")
    elif 200 < flux <= 400:
        colours.append("#ffb299")
    elif 400 < flux <= 600:
        colours.append("#ff8969")
    elif 600 < flux <= 800:
        colours.append("#ff5b3a")
    elif 800 < flux <= 1000:
        colours.append("#ff0000")

In [73]:
len(colours)

4200

In [74]:
perturbed_dataframe = pd.DataFrame({"fluxes": perturbed_flux, "colours": colours})
perturbed_dataframe.index = reaction_ids
perturbed_dataframe

,fluxes,colours
RCR10001,0.0,#ffffff
RCR10002,0.0,#ffffff
RCR10004,-500.0,#a074ff
RCR10005,0.0,#ffffff
RCR10006,0.0,#ffffff
...,...,...
RCR90207,0.0,#ffffff
RCR90208,0.0,#ffffff
RCR90209,0.0,#ffffff
RCR90210,0.0,#ffffff


In [100]:
nodes_perturbed = []
links_perturbed = []
pseudo_nodes_s_perturbed = []
pseudo_nodes_e_perturbed = []
selector_perturbed = "s"
selected_list_of_reactions_perturbed = []

for item in nodes_FA:
    if item["group"] == selector_perturbed:
        nodes_perturbed.append(item)

for item in fatty_acid["Reactions"]:
    for reactant in item["Reactants"]:
        if reactant["group"] == selector_perturbed and item not in selected_list_of_reactions_perturbed:
                selected_list_of_reactions_perturbed.append(item)
    if "Products" in item:
        for product in item["Products"]:
            if product["group"] == selector_perturbed and item not in selected_list_of_reactions_perturbed:
                selected_list_of_reactions_perturbed.append(item)

for item in selected_list_of_reactions_perturbed:
    if "Products" in item:
        pseudo_nodes_s_perturbed.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"})
        pseudo_nodes_e_perturbed.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
        for reactant in item["Reactants"]:
            links_perturbed.append({"source": reactant["id"], "target": item["id"] + "s", "arrow": False, "colour": perturbed_dataframe.loc[item["id"], "colours"]})
            if reactant not in nodes_perturbed:
                nodes_perturbed.append(reactant)
        for product in item["Products"]:
            links_perturbed.append({"source": item["id"] + "e", "target": product["id"], "arrow": True, "colour": perturbed_dataframe.loc[item["id"], "colours"]})
            if product not in nodes_perturbed:
                nodes_perturbed.append(product)

for index, item in enumerate(pseudo_nodes_s_perturbed):
    links_perturbed.append({"source": item["id"], "target": pseudo_nodes_e_perturbed[index]["id"], "arrow": False, "colour": perturbed_dataframe.loc[item["id"].replace("s", "").replace("e", ""), "colours"]})

all_nodes_perturbed = nodes_perturbed + pseudo_nodes_s_perturbed + pseudo_nodes_e_perturbed

print(len(nodes_perturbed))
print(len(pseudo_nodes_s_perturbed))
print(len(pseudo_nodes_e_perturbed))
print(len(all_nodes_perturbed))
print(len(links_perturbed))
print(len(selected_list_of_reactions_perturbed))


505
888
888
2281
4901
1041


In [101]:
icardio_fatty_acid_s_perturbed= {"nodes": [], "links": []}
icardio_fatty_acid_s_perturbed["nodes"] += all_nodes_perturbed
icardio_fatty_acid_s_perturbed["links"] += links_perturbed

In [102]:
json_string = json.dumps(icardio_fatty_acid_s_perturbed)

with open('icardio_fatty_acid_s_perturbed.json', 'w', encoding='utf-8') as f:
    json.dump(icardio_fatty_acid_s_perturbed, f, ensure_ascii=False, indent=4)